In [50]:
import os
import openai
from dotenv import load_dotenv
# Load environment variables
load_dotenv()
import json

class MainChat:
    def __init__(self, system_message):
        self.api_type = "azure"
        self.api_base = "https://decmasterthesis25.openai.azure.com/"
        self.api_version = "2023-07-01-preview"
        self.api_key = os.getenv("AZURE_OPENAI_API_KEY")
        
        openai.api_key = self.api_key
        openai.api_type = self.api_type
        openai.api_base = self.api_base
        openai.api_version = self.api_version

        self.system_message = system_message
        
        self.chat_history = []
        self.chat_history.append({"role": "system", "content": system_message})

    def chat(self, user_input, system_message = None):
        try:
            message_text = {"role": "user", "content": user_input}
            self.chat_history.append(message_text)

            completion = openai.ChatCompletion.create(
                engine="gpt-4",
                messages=self.chat_history,
                temperature=0.7,
                max_tokens=2000,
                top_p=0.95,
                frequency_penalty=0,
                presence_penalty=0,
                stop=None
            )
            
            response_content = completion.choices[0].message['content'].strip()
            self.chat_history.append({"role": "system", "content": response_content})
        
            return response_content
        except Exception as e:
            return f"An error occurred: {str(e)}"
    def reset_history(self):
        self.chat_history = []
        self.chat_history.append({"role": "system", "content": self.system_message})




In [18]:
# Example usage
system_message_agent_1 = """
We are playing a simulation game. You are a helper searching for terminology not in your terminology list.
You are only allowed to speak with the instructions given below and nothing else. 

You have limited terminology knowledge, but you can learn new terminology. 
Imagine know have a TERMINOLOGY-LIST of a typical 16 year old teenager as database.
Below you will get extra terminology you know in addition to the terminology of a typical 16 year old teenager.
If words are not existent in the TERMINOLOGY-LIST, you answer with "DONT UNDERSTAND:" followed with the list. 
and give a list of words why you are answering with 'DONT UNDERSTAND:'.

IF you get an explanation of the new terms, analyze the text (both the term and the explanation) for new terms that are not in your TERMINOLOGY-LIST.
IF all terms in the text (including in the explanation) are in the TERMINOLOGY-LIST, list all new-learned terms and end with TERMINATE as last word
ELSE you answer with "DONT UNDERSTAND:" followed with the list of words not in your TERMINOLOGY-LIST.
If a verb is in the TERMINOLOGY-LIST, you also know the conjugations.

Extra words you know: """

chat_instance = MainChat(system_message_agent_1)

system_message_explainer = """You are a helpful AI Assistant that explains words. You explain all terms after 'DONT UNDERSTAND: *a list of words*',, 
THEN you should answer with explanations for those words in less than 2 sentences. List each word.
IF you understand everything, then asnwer with 'quit'
"""

chat_instance_explainer = MainChat(system_message_explainer)

user_input = 'What is reinforcement learning in AI?'
response = chat_instance.chat(user_input.strip())
explained = chat_instance_explainer.chat(response.strip())

print("Term-AI:", response)
print("Explainer-AI:", explained)

Term-AI: DONT UNDERSTAND: reinforcement, AI
Explainer-AI: Reinforcement: It is a term in psychology which refers to the process of using rewards or punishment to encourage or discourage certain behaviors. In the context of machine learning, it's a type of learning where an agent learns to behave in an environment, by performing certain actions and observing the results.

AI: AI stands for Artificial Intelligence. It is a branch of computer science dealing with the simulation of intelligent behavior in computers, meaning it involves creating machines or software that can imitate human intelligence.


In [19]:
user_input = explained
response = chat_instance.chat(user_input.strip())
print("Term-AI:\n", response)



Term-AI:
 NEW TERMS LEARNED: 
Reinforcement, AI
TERMINATE


Example 2

In [21]:
chat_instance = MainChat(system_message_agent_1)
chat_instance_explainer = MainChat(system_message_explainer)

user_input = 'Backpropagation, or the backward propagation of errors, is a quintessential algorithm underpinning the training mechanism of neural networks, facilitating the optimization of weights through meticulous iterative adjustments. This algorithm operates on the principle of gradient descent, leveraging the calculus concept of partial derivatives to compute the gradient of the loss function with respect to each weight in the network.'
response = chat_instance.chat(user_input.strip())

print("Term-AI:\n", response)




Term-AI: DONT UNDERSTAND: quintessential, underpinning, facilitating, optimization, meticulous, iterative, adjustments, principle, gradient, descent, leveraging, calculus, partial derivatives, compute, gradient, loss function.


In [22]:
explained = chat_instance_explainer.chat(response.strip())
print("Explainer-AI:", explained)


Explainer-AI: 1. Quintessential: Representing the most perfect or typical example of a quality or class.
2. Underpinning: A solid foundation or basis of a structure or argument.
3. Facilitating: Making an action or process easier or less difficult.
4. Optimization: The process of making something as fully perfect, functional, or effective as possible.
5. Meticulous: Showing great attention to detail; very careful and precise.
6. Iterative: Involving repetition, as in a process or sequence.
7. Adjustments: Changes or alterations designed to produce a desired effect, fit, or result.
8. Principle: A fundamental truth or proposition that serves as the foundation for a system of belief or behavior.
9. Gradient: A measure of the steepness of a graph at any point.
10. Descent: The action of moving downward, dropping, or falling.
11. Leveraging: Using something to maximum advantage.
12. Calculus: A branch of mathematics that deals with rates of change and the accumulation of quantities.
13. Pa

In [23]:
user_input = explained
response = chat_instance.chat(user_input.strip())
print("Term-AI:\n", response)

Term-AI:
 NEW-LEARNED TERMS: quintessential, underpinning, facilitating, optimization, meticulous, iterative, adjustments, principle, gradient, descent, leveraging, calculus, partial derivatives, compute, gradient, loss function. TERMINATE


example 3

In [35]:

system_message_agent_1 = """
We are playing a simulation game. You are a helper searching for terminology not in your terminology list.
You are only allowed to speak with the instructions given below and nothing else. 

You have limited terminology knowledge, but you can learn new terminology. 
Imagine you have a TERMINOLOGY-LIST of a typical 16 year old teenager as database.

If words are not existent in your TERMINOLOGY-LIST, you answer with "DONT UNDERSTAND:" followed with the list. 
If you get an explanation for a word, analyze the explanation and do the same as above.
If no new words are present, answer with TERMINATE'.
"""

system_message_explainer = """You are a helpful AI Assistant that explains words. You explain all terms after 'DONT UNDERSTAND: *a list of words*',, 
THEN you should answer with explanations for those words in less than 4 sentences. List each word.
IF you understand everything, then asnwer with 'quit'
"""

chat_instance = MainChat(system_message_agent_1)
chat_instance_explainer = MainChat(system_message_explainer)

user_input = 'What is deep learning?'
response = chat_instance.chat(user_input.strip())

print("Term-AI:\n", response)

Term-AI:
 DONT UNDERSTAND: deep learning


In [36]:
explained = chat_instance_explainer.chat(response.strip())
print("Explainer-AI:", explained)

Explainer-AI: Deep learning is a subset of machine learning, a type of artificial intelligence (AI) that involves the creation and use of neural networks with several layers. These layers are a kind of modelled inspired by the human brain and are designed to translate inputs into outputs that the programmer wants. It's called "deep" because of the potentially many layers that can be in the neural network.


In [37]:
user_input = explained
response = chat_instance.chat(user_input.strip())
print("Term-AI:\n", response)

Term-AI:
 DONT UNDERSTAND: machine learning, artificial intelligence, neural networks, modelled, inputs, outputs, programmer, "deep"


In [38]:
explained = chat_instance_explainer.chat(response.strip())
print("Explainer-AI:", explained)

Explainer-AI: 1. Machine Learning: A field of artificial intelligence that uses statistical techniques to enable computer systems to 'learn' from data without being explicitly programmed.
2. Artificial Intelligence: The theory and development of computer systems able to perform tasks normally requiring human intelligence, such as visual perception, speech recognition, decision-making, and translation.
3. Neural Networks: A series of algorithms that attempts to identify underlying relationships in a set of data through a process that mimics the way the human brain operates.
4. Modelled: In computing, it refers to the process of creating a representation of a system or a specific situation in order to simulate or predict its behavior.
5. Inputs: The data that is fed into a system, machine, or model.
6. Outputs: The results produced by a system, machine, or model.
7. Programmer: A person who writes and tests computer code, which can be used to create or control software and applications.


In [39]:
user_input = explained
response = chat_instance.chat(user_input.strip())
print("Term-AI:\n", response)

Term-AI:
 TERMINATE'


In [41]:

system_message_agent_1 = """
We are playing a simulation game. You are a helper searching for terminology not in your terminology list.
You are only allowed to speak with the instructions given below and nothing else. 

You have limited terminology knowledge, but you can learn new terminology. 
Imagine you have a TERMINOLOGY-LIST of a typical 16 year old teenager as database.

If words are not existent in your TERMINOLOGY-LIST, you answer with "DONT UNDERSTAND:" followed with the list. 
If you get an explanation for a word, analyze the explanation and do the same as above.
If no new words are present, answer with TERMINATE'.
"""

system_message_explainer = """You are a helpful AI Assistant that explains words. You explain all terms after 'DONT UNDERSTAND: *a list of words*',, 
THEN you should answer with explanations for those words in less than 10 sentences. List each word.
IF you understand everything, then asnwer with 'quit'
"""

chat_instance = MainChat(system_message_agent_1)
chat_instance_explainer = MainChat(system_message_explainer)

user_input = 'deep learning?'
response = chat_instance.chat(user_input.strip())

print("Term-AI:\n", response)

Term-AI:
 DONT UNDERSTAND: deep learning


In [42]:
explained = chat_instance_explainer.chat(response.strip())
print("Explainer-AI:", explained)

Explainer-AI: Deep learning is a type of artificial intelligence (AI) that mimics the workings of the human brain in processing data for use in decision making. It's a subset of machine learning in AI that has networks which are capable of learning unsupervised from data that is unstructured or unlabeled. Deep learning is used in many industries for various tasks such as voice recognition, image recognition, and natural language processing.


In [43]:
user_input = explained
response = chat_instance.chat(user_input.strip())
print("Term-AI:\n", response)

Term-AI:
 DONT UNDERSTAND: artificial intelligence, machine learning, networks, unsupervised learning, unstructured data, unlabeled data, voice recognition, image recognition, natural language processing.


In [44]:
explained = chat_instance_explainer.chat(response.strip())
print("Explainer-AI:", explained)

Explainer-AI: 1. Artificial Intelligence (AI): This refers to the simulation of human intelligence processes by machines, especially computer systems. AI can include learning, reasoning, and self-correction.

2. Machine Learning: A type of AI that gives computers the ability to learn from and make decisions without being explicitly programmed. It uses algorithms and statistical models to perform tasks without explicit instructions, relying on patterns and inference instead.

3. Networks: In the context of AI and Machine Learning, networks often refer to 'neural networks,' which are computing systems inspired by the human brain. They are interconnected layers of algorithms, called neurons, that feed data into each other, and they can learn from data.

4. Unsupervised Learning: A type of machine learning algorithm used to draw inferences from datasets consisting of input data without labeled responses. The most common unsupervised learning method is cluster analysis.

5. Unstructured Dat

In [45]:
user_input = explained
response = chat_instance.chat(user_input.strip())
print("Term-AI:\n", response)

Term-AI:
 TERMINATE'


In [46]:
system_message_agent_1 = """
We are playing a simulation game. You are a helper searching for terminology not in your terminology list.
You are only allowed to speak with the instructions given below and nothing else. 

You have limited terminology knowledge, but you can learn new terminology. 
Imagine you have a TERMINOLOGY-LIST of a typical 16 year old teenager as database.

If words are not existent in your TERMINOLOGY-LIST, you answer with "DONT UNDERSTAND:" followed with the list. 
If you get an explanation for a word, analyze the explanation and do the same as above.
If no new words are present, answer with TERMINATE'.
"""

system_message_explainer = """You are a helpful AI Assistant that explains words. You explain all terms after 'DONT UNDERSTAND: *a list of words*',, 
THEN you should answer with explanations for those words in less than 10 sentences. List each word.
IF you understand everything, then asnwer with 'quit'
"""

chat_instance = MainChat(system_message_agent_1)
chat_instance_explainer = MainChat(system_message_explainer)

user_input = 'unsupervised learning'
response = chat_instance.chat(user_input.strip())

print("Term-AI:\n", response)

Term-AI:
 DONT UNDERSTAND: unsupervised learning


In [47]:
explained = chat_instance_explainer.chat(response.strip())
print("Explainer-AI:", explained)

Explainer-AI: Unsupervised learning is a type of machine learning that looks for previously undetected patterns in a data set with no pre-existing labels and with a minimum of human supervision. In contrast to supervised learning that usually makes use of human-labeled data, unsupervised learning, also known as self-organization allows for modeling of probability densities of given inputs. It's commonly used in applications like anomaly detection, association rule mining, and clustering.


In [48]:
user_input = explained
response = chat_instance.chat(user_input.strip())
print("Term-AI:\n", response)

Term-AI:
 DONT UNDERSTAND: machine learning, patterns, data set, pre-existing labels, human supervision, supervised learning, human-labeled data, self-organization, modeling of probability densities, inputs, anomaly detection, association rule mining, clustering.


In [49]:
explained = chat_instance_explainer.chat(response.strip())
print("Explainer-AI:", explained)

Explainer-AI: 1. Machine Learning: A field of artificial intelligence that uses statistical techniques to enable computer systems to learn and improve from experience.

2. Patterns: In the context of data analysis, patterns refer to the recurring or regular arrangements in the data.

3. Data Set: A collection of related sets of information that is composed of separate elements but can be manipulated as a unit by a computer.

4. Pre-existing Labels: Information associated with data that defines its characteristics or category.

5. Human Supervision: The process where humans oversee, guide, or manage a process or system.

6. Supervised Learning: A type of machine learning where models are trained using labeled data, i.e., data paired with the correct output.

7. Human-labeled Data: Data that has been categorized or tagged with relevant labels by humans.

8. Self-Organization: A process where some form of overall order or coordination arises out of the local interactions between parts of 

In [51]:
user_input = explained
response = chat_instance.chat(user_input.strip())
print("Term-AI:\n", response)

Term-AI:
 TERMINATE'


In [53]:
chat_instance = MainChat(system_message_agent_1)
chat_instance_explainer = MainChat(system_message_explainer)

user_input = 'machine learning'
response = chat_instance.chat(user_input.strip())
print("Term-AI:\n", response)

explained = chat_instance_explainer.chat(response.strip())
print("Explainer-AI:\n", explained)

user_input = explained
response = chat_instance.chat(user_input.strip())
print("Term-AI:\n", response)

explained = chat_instance_explainer.chat(response.strip())
print("Explainer-AI:\n", explained)

user_input = explained
response = chat_instance.chat(user_input.strip())
print("Term-AI:\n", response)

Term-AI:
 DONT UNDERSTAND: 'machine learning'
Explainer-AI: Machine learning is a type of artificial intelligence (AI) that allows a computer system to learn from data input and improve from experience, without being specifically programmed to do so. In other words, it's a method of data analysis that automates the building of analytical models. It is used in various sectors to make predictions or decisions without human intervention.
Term-AI:
 DONT UNDERSTAND: 'artificial intelligence', 'computer system', 'data input', 'experience', 'method', 'data analysis', 'automates', 'analytical models', 'sectors', 'predictions', 'decisions', 'human intervention'
Explainer-AI: 1. Artificial Intelligence: It refers to the simulation of human intelligence processes by machines, especially computer systems. This involves learning, reasoning, and self-correction.
2. Computer System: A computer system is a set of integrated devices that input, output, process, and store data and information.
3. Data I

In [57]:

chat_instance = MainChat(system_message_agent_1)
chat_instance_explainer = MainChat(system_message_explainer)

user_input = 'Artificial Intelligence (AI), Neural Networks, Machine Learning, Supervised Learning, Unsupervised Learning, Reinforcement Learning, Convolutional Neural Networks (CNNs), Recurrent Neural Networks (RNNs), Deep Neural Networks (DNNs), Backpropagation, Feature Extraction, Activation Function, Loss Function, Gradient Descent, Tensor Processing Units (TPUs), Graphics Processing Units (GPUs), Overfitting, Underfitting, Regularization, Data Augmentation, Transfer Learning, Natural Language Processing (NLP), Computer Vision, Generative Adversarial Networks (GANs), Autoencoders'
response = chat_instance.chat(user_input.strip())
print("Term-AI:\n", response)

explained = chat_instance_explainer.chat(response.strip())
print("Explainer-AI:\n", explained)

user_input = explained
response = chat_instance.chat(user_input.strip())
print("Term-AI:\n", response)

explained = chat_instance_explainer.chat(response.strip())
print("Explainer-AI:\n", explained)

user_input = explained
response = chat_instance.chat(user_input.strip())
print("Term-AI:\n", response)

Term-AI:
 DONT UNDERSTAND: Artificial Intelligence (AI), Neural Networks, Machine Learning, Supervised Learning, Unsupervised Learning, Reinforcement Learning, Convolutional Neural Networks (CNNs), Recurrent Neural Networks (RNNs), Deep Neural Networks (DNNs), Backpropagation, Feature Extraction, Activation Function, Loss Function, Gradient Descent, Tensor Processing Units (TPUs), Graphics Processing Units (GPUs), Overfitting, Underfitting, Regularization, Data Augmentation, Transfer Learning, Natural Language Processing (NLP), Computer Vision, Generative Adversarial Networks (GANs), Autoencoders.
Explainer-AI:
 1. Artificial Intelligence (AI): AI is the simulation of human intelligence processes by machines, especially computer systems.
2. Neural Networks: Neural networks are a set of algorithms, modeled loosely after the human brain, designed to recognize patterns.
3. Machine Learning: Machine Learning is a type of AI that allows a system to learn and improve from experience without 